In [1]:
import os
import pandas as pd
import cv2  # type: ignore
import IPython.display as display  # type: ignore
from PIL import Image  # type: ignore
from io import BytesIO
import time

In [2]:
# Изменение текущей рабочей директории
os.chdir("../")
%pwd

'/Users/andrejizvarin/Documents/___project/emotion_recognition'

In [3]:
import warnings
from urllib3.exceptions import InsecureRequestWarning  # type: ignore
from src.face_detector import FaceDetector
from src.emotion_classifier import EmotionClassifier

# Подавление всех предупреждений от библиотеки urllib3
warnings.simplefilter("ignore", category=InsecureRequestWarning)

# Инициализация классов для детектирования лиц и классификации эмоций
face_detector = FaceDetector()
emotion_classifier = EmotionClassifier()

/Users/andrejizvarin/Documents/___project/emotion_recognition/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Путь к папке с тестовыми изображениями и файлу с ожидаемыми результатами
test_images_dir = "data/test_kaggle"
sample_submission_path = "data/sample_submission.csv"

# Загрузка ожидаемых результатов
expected_results = pd.read_csv(sample_submission_path)

# Проход по всем изображениям в папке тестовых изображений
for filename in os.listdir(test_images_dir):
    if filename.endswith(".jpg"):
        image_path = os.path.join(test_images_dir, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        faces = face_detector.detect_faces(image)
        for x, y, w, h in faces:
            face_img = image[y : y + h, x : x + w]
            emotion_name = emotion_classifier.predict_emotion(face_img)

            # Результат предсказания
            print(f"File: {filename}, Predicted emotion: {emotion_name}")

            # Поиск ожидаемой эмоции для изображения
            expected_emotion = expected_results.loc[
                expected_results["image_path"] == filename, "emotion"
            ].iloc[0]
            print(f"Expected emotion: {expected_emotion}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 514ms/step
File: 63.jpg, Predicted emotion: contempt
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
File: 823.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
File: 4217.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
File: 3578.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
File: 2666.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
File: 2100.jpg, Predicted emotion: disgust
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
File: 2100.jpg, Predicted emotion: contempt
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
File: 189.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
File: 2114.jpg, Predicted emotion: uncertain
Expected emotion: neutral
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

In [5]:
def show_array(a, fmt="JPEG"):
    f = BytesIO()
    Image.fromarray(a).save(f, fmt)
    display.display(display.Image(data=f.getvalue()))


cap = cv2.VideoCapture(0)  # Запуск захвата видео с веб-камеры

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        faces = face_detector.detect_faces(frame)

        for x, y, w, h in faces:
            face_img = frame[y : y + h, x : x + w]
            emotion_name = emotion_classifier.predict_emotion(face_img)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(
                frame,
                emotion_name,
                (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.9,
                (255, 255, 255),
                2,
            )

        show_array(frame)
        time.sleep(0.1)  # Короткая пауза перед следующим кадром
        display.clear_output(wait=True)
except KeyboardInterrupt:
    print("Прервано пользователем.")
finally:
    cap.release()  # Освобождение ресурсов камеры

Прервано пользователем.
